In [2]:
cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/

/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/Train_FeatureEngineering_3.csv")
print("Shape of Training Data",train.shape)
test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/Test_FeatureEngineering_3.csv")
print("Shape of Testing Data", test.shape)


Shape of Training Data (16844, 24)
Shape of Testing Data (8576, 23)


In [5]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'trip_distance'],
      dtype='object')

In [6]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'trip_distance'],
      dtype='object')

# Data_Preprocessing

In [0]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_date']
drop_columns_test=['tripid','pickup_time','drop_time','pickup_date']

In [8]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (16844, 20)
Shape of Testing Data after dropping columns (8576, 19)


In [9]:
train_1=pd.get_dummies(train_1)
test_1=pd.get_dummies(test_1)

print("Shape of Training Data after One Hot Encoding",train_1.shape)
print("Shape of Testing Data after One Hot Encoding",test_1.shape)

Shape of Training Data after One Hot Encoding (16844, 20)
Shape of Testing Data after One Hot Encoding (8576, 19)


In [10]:
print('Old size: %d' % len(train))
train_1 = train_1.dropna()
print('New size: %d' % len(train))

Old size: 16844
New size: 16844


In [0]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [13]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13475
Number of records in validation data  3369


# XGBoost CLassifier

In [22]:
model = XGBClassifier()
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [21]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.51%


In [0]:
y_pred = model.predict(test_1)
predictions = [round(value) for value in y_pred]

# Linear Regression Classifier

In [0]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=lm.predict(X_test)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)


RMSE for Linear Regression is  0.2939505199843409


In [0]:
y_pred=lm.predict(test_1)

# LGBM Classifier

In [0]:
def objective(space):
clf = lgb.LGBMRegressor(
          objective = 'regression',
          n_jobs = -1, # Updated from 'nthread'
          verbose=1,
          boosting_type='gbdt',
        num_leaves=60,
        bagging_freq=20,
       subsample_freq=100,
    max_depth=int(space['max_depth']),
    subsample=space['subsample'],
        n_estimators=5000,
    colsample_bytree=space['colsample'])
          #metric='l2_root')
eval_set=[( X_train, y_train), ( X_test,y_test)]
clf.fit(X_train, np.array(y_train),
            eval_set=eval_set,eval_metric='rmse',
            early_stopping_rounds=20)
pred = clf.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    print("SCORE:", rmse)

# Get Predictions

In [0]:
 y_pred = pd.DataFrame({'prediction':predictions})
 y_pred["prediction"] = y_pred["prediction"].astype(int)
 y_pred['prediction'] = y_pred['prediction'].apply(lambda x: 0 if x < 0.5 else 1)
 y_pred.to_csv("predictions/predications_xgboost_6.csv")